<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating GANs with Desired Features

In this notebook I show you you can use GANs to create faces that contain features that you desire.  In this case I wanted to create a Kaggle competition for my students to detect if faces had glasses or not.  This turned out to be a fairly easy Kaggle, but I think the process I used to generate faces with glasses or not is interesting.  You can see the Kaggle here.

* [Glasses or Not Kaggle](https://www.kaggle.com/c/applications-of-deep-learningwustl-spring-2020/overview)

I used Google CoLab to run this, a GPU is required.

You can see a YouTube video on this notebook:

* [Controlling if GANs Have Glasses or Not](https://www.kaggle.com/c/applications-of-deep-learningwustl-spring-2020/overview)

In [1]:
# Run this for Google CoLab (use TensorFlow 1.x)
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

TensorFlow 1.x selected.
Mounted at /content/drive


Next, clone StyleGAN2 from GitHub.

In [2]:
#!git clone https://github.com/NVlabs/stylegan2.git
!git clone https://github.com/mit-han-lab/data-efficient-gans.git

Cloning into 'data-efficient-gans'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 596 (delta 10), reused 16 (delta 6), pack-reused 575
Receiving objects: 100% (596/596), 23.09 MiB | 39.14 MiB/s, done.
Resolving deltas: 100% (330/330), done.


Verify that StyleGAN has been cloned.

In [4]:
!ls /content/data-efficient-gans/DiffAugment-stylegan2/

DiffAugment_tf.py  generate_gif.py  metrics    run_cifar.py  run_low_shot.py
dnnlib		   LICENSE.txt	    README.md  run_ffhq.py   training


# Run StyleGAN2 From Python Code

Add the StyleGAN folder to Python so that you can import it.  The code below is based on code from NVidia. This actually generates your images.

In [5]:
import sys
#sys.path.insert(0, "/content/stylegan2")
sys.path.insert(0, "/content/data-efficient-gans/DiffAugment-stylegan2")

import dnnlib

In [10]:
# Copyright (c) 2019, NVIDIA Corporation. All rights reserved.
#
# This work is made available under the Nvidia Source Code License-NC.
# To view a copy of this license, visit
# https://nvlabs.github.io/stylegan2/license.html

import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

#import pretrained_networks
from training import misc
#----------------------------------------------------------------------------

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

def generate_images(Gs, seeds, truncation_psi, prefix):
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
        rnd = np.random.RandomState(0)
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
        path = f"/content/{prefix}-{seed_idx+1}.png"
        PIL.Image.fromarray(images[0], 'RGB').save(path)


In [14]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = '/content/drive/MyDrive/torilarit/network-snapshot-000400.pkl'

print('Loading networks from "%s"...' % network_pkl)
tflib.init_tf({'rnd.np_random_seed': 1000})
_, _, Gs = misc.load_pkl(network_pkl)
vector_size = Gs.input_shape[1:][0]

Loading networks from "/content/drive/MyDrive/torilarit/network-snapshot-000400.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.


# Explore Many GANs

The first thing I did was generate many different GAN's randomly.  I am going to choose from those to find ones with the traits that I am looking for.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

# 8227
for i in range(1000):
  z = 9200+i
  print(z)
  seeds = expand_seed([z], vector_size)
  generate_images(Gs, seeds, 0.5, "image")
  img = cv2.imread('/content/image-1.png')   
  cv2_imshow(img) 

# Modify Single GAN

This section I used mainly to generate the art for the Kaggle.  I wanted to see if I could generate the same person both with and without glasses.  I modified the latent vector until I was able to see the glasses vanish.  Then I minimized the change to the vector until the glasses returned.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

seeds = expand_seed([9220], vector_size)

generate_images(Gs, seeds, 0.5, "image")
img = cv2.imread('/content/image-1.png')   
cv2_imshow(img) 

# Explore Elements of Vector

This section of code let me explore 10-block sections of the latent vector looking for what part might affect the attribute I desire.

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

i = 0

while i<500:
  seeds = expand_seed([9223], vector_size)
  seeds[0][0][i:i+10] = -2
  print(i)
  generate_images(Gs, seeds, 0.5, "image")
  img = cv2.imread('/content/image-1.png')   
  cv2_imshow(img) 
  i+=10

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

i = 0

seeds = expand_seed([9223], vector_size)
#seeds[0][0][410] = -2
#seeds[0][0][411] = -2
seeds[0][0][412] = -2
#seeds[0][0][413] = -2
seeds[0][0][414] = -2
#seeds[0][0][415] = -2
#seeds[0][0][416] = -2
#seeds[0][0][417] = -2
seeds[0][0][418] -= 1.3
#seeds[0][0][419] = -2
print(i)
generate_images(Gs, seeds, 0.5, "image")
img = cv2.imread('/content/image-1.png')   
cv2_imshow(img) 
i+=1

# What is an Image??

In [18]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = '/content/drive/MyDrive/torilarit/network-snapshot-000400.pkl'

print('Loading networks from "%s"...' % network_pkl)
tflib.init_tf({'rnd.np_random_seed': 1000})
_, _, Gs = misc.load_pkl(network_pkl)
vector_size = Gs.input_shape[1:][0]

Loading networks from "/content/drive/MyDrive/torilarit/network-snapshot-000400.pkl"...


In [35]:
GLASSES = [9268, 9282, 9286, 9297, 9317, 9321, 9324, 9328, 9346, 9348, 9351, 9355, 9375, 9379, 9408, 9424, 9486, 9489, 9490, 9496, 9499]

NO_GLASSES = [x+9200 for x in range(300) if x+9200 not in GLASSES]
print (NO_GLASSES)
#GLASSES er menn

[9200, 9201, 9202, 9203, 9204, 9205, 9206, 9207, 9208, 9209, 9210, 9211, 9212, 9213, 9214, 9215, 9216, 9217, 9218, 9219, 9220, 9221, 9222, 9223, 9224, 9225, 9226, 9227, 9228, 9229, 9230, 9231, 9232, 9233, 9234, 9235, 9236, 9237, 9238, 9239, 9240, 9241, 9242, 9243, 9244, 9245, 9246, 9247, 9248, 9249, 9250, 9251, 9252, 9253, 9254, 9255, 9256, 9257, 9258, 9259, 9260, 9261, 9262, 9263, 9264, 9265, 9266, 9267, 9269, 9270, 9271, 9272, 9273, 9274, 9275, 9276, 9277, 9278, 9279, 9280, 9281, 9283, 9284, 9285, 9287, 9288, 9289, 9290, 9291, 9292, 9293, 9294, 9295, 9296, 9298, 9299, 9300, 9301, 9302, 9303, 9304, 9305, 9306, 9307, 9308, 9309, 9310, 9311, 9312, 9313, 9314, 9315, 9316, 9318, 9319, 9320, 9322, 9323, 9325, 9326, 9327, 9329, 9330, 9331, 9332, 9333, 9334, 9335, 9336, 9337, 9338, 9339, 9340, 9341, 9342, 9343, 9344, 9345, 9347, 9349, 9350, 9352, 9353, 9354, 9356, 9357, 9358, 9359, 9360, 9361, 9362, 9363, 9364, 9365, 9366, 9367, 9368, 9369, 9370, 9371, 9372, 9373, 9374, 9376, 9377, 9378, 938

In [36]:
import random
import pandas as pd

GEN_SET = GLASSES

def generate_set(GEN_SET, count):
  seeds3 = []

  for i in range(count):
    i1 = random.randint(0,len(GEN_SET)-1)
    i2 = random.randint(0,len(GEN_SET)-1)
    seeds1 = expand_seed([GEN_SET[i1]],vector_size)
    seeds2 = expand_seed([GEN_SET[i2]],vector_size)
    seeds = [seeds1[0][0], seeds2[0][0]]

    diff = seeds[1] - seeds[0]
    delta = (np.random.rand(512) * 0.6) + .2
    step = diff * delta
    current = [seeds[0].copy()]
    current += step
    current = np.round(current,5)
    seeds3.append(current)

  seeds4 = [x[0] for x in seeds3]
  seeds4 = np.array(seeds4)
  return pd.DataFrame(seeds4)



#generate_images(Gs, seeds3,0.5,'glasses')

df1 = generate_set(GLASSES, int(5000 * .64))
df1['glasses'] = 1
df2 = generate_set(NO_GLASSES, int(5000 * .36))
df2['glasses'] = 0

In [37]:
df = pd.concat([df1,df2],ignore_index=True)

In [38]:
v_cols2 = [f'v{i+1}' for i in range(512)]
cols2 = v_cols2.copy()
cols2.append('glasses')
df.columns = cols2
df = df.reindex(np.random.permutation(df.index))
df.insert(0, 'id', range(1,len(df)+1))

In [39]:
l = len(df)
sz = int(l*.9)
df_train = df[0:sz].copy()
df_test = df[sz:].copy()
df_test.drop('glasses',axis=1,inplace=True)
df_solution = df[sz:].copy()
df_sample = df_test.copy()
df_sample['glasses'] = 0.64

In [40]:
df_train.to_csv('/content/train.csv',index=False)
df_test.to_csv('/content/test.csv',index=False)
df_solution.to_csv('/content/solution.csv',index=False)
df_sample.to_csv('/content/sample.csv',index=False)


# Build Images

In [41]:
df.shape

(5000, 514)

In [42]:
seeds = df[v_cols2].values
seeds = [x.reshape([1,512]) for x in seeds]
generate_images(Gs, seeds,0.5,"face")

Generating image for seed 0/5000 ...
Generating image for seed 1/5000 ...
Generating image for seed 2/5000 ...
Generating image for seed 3/5000 ...
Generating image for seed 4/5000 ...
Generating image for seed 5/5000 ...
Generating image for seed 6/5000 ...
Generating image for seed 7/5000 ...
Generating image for seed 8/5000 ...
Generating image for seed 9/5000 ...
Generating image for seed 10/5000 ...
Generating image for seed 11/5000 ...
Generating image for seed 12/5000 ...
Generating image for seed 13/5000 ...
Generating image for seed 14/5000 ...
Generating image for seed 15/5000 ...
Generating image for seed 16/5000 ...
Generating image for seed 17/5000 ...
Generating image for seed 18/5000 ...
Generating image for seed 19/5000 ...
Generating image for seed 20/5000 ...
Generating image for seed 21/5000 ...
Generating image for seed 22/5000 ...
Generating image for seed 23/5000 ...
Generating image for seed 24/5000 ...
Generating image for seed 25/5000 ...
Generating image for s

KeyboardInterrupt: ignored

In [43]:
!zip /content/drive/My\ Drive/torilarit/images.zip /content/*

updating: content/data-efficient-gans/ (stored 0%)
updating: content/drive/ (stored 0%)
updating: content/face-100.png (deflated 0%)
updating: content/face-101.png (deflated 0%)
updating: content/face-102.png (deflated 0%)
updating: content/face-103.png (deflated 0%)
updating: content/face-104.png (deflated 0%)
updating: content/face-105.png (deflated 0%)
updating: content/face-106.png (deflated 0%)
updating: content/face-107.png (deflated 0%)
updating: content/face-108.png (deflated 0%)
updating: content/face-109.png (deflated 0%)
updating: content/face-10.png (deflated 0%)
updating: content/face-110.png (deflated 0%)
updating: content/face-111.png (deflated 0%)
updating: content/face-112.png (deflated 0%)
updating: content/face-113.png (deflated 0%)
updating: content/face-114.png (deflated 0%)
updating: content/face-115.png (deflated 0%)
updating: content/face-116.png (deflated 0%)
updating: content/face-117.png (deflated 0%)
updating: content/face-118.png (deflated 0%)
updating: con

In [44]:
!ls -l /content/drive/My\ Drive/torilarit/images.zip

-rw------- 1 root root 56445736 Dec 20 16:25 '/content/drive/My Drive/torilarit/images.zip'


In [33]:
!ls -l /content/*.csv

-rw-r--r-- 1 root root  2156285 Dec 20 16:07 /content/sample.csv
-rw-r--r-- 1 root root  2154785 Dec 20 16:07 /content/solution.csv
-rw-r--r-- 1 root root  2153777 Dec 20 16:07 /content/test.csv
-rw-r--r-- 1 root root 19372120 Dec 20 16:07 /content/train.csv
